<a href="https://colab.research.google.com/github/jmontano1987/pdg/blob/main/Proyecto_de_Grado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Instalaciones**

In [4]:
!pip install tweepy
!pip3 install tqdm
!pip install matplotlib
!pip install wordcloud

#**Librerias**

In [15]:
import io
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np
import string
import tweepy
from tqdm.notebook import tqdm as tqdm_notebook
from dateutil import tz
from datetime import datetime 

#**Funciones**

In [36]:
def convert_date(date):
  str_format_date = '%Y-%m-%d %H:%M:%S' 
  from_zone = tz.gettz('UTC')
  to_zone = tz.gettz('America/Bogota')
  d = datetime.strptime("2022-02-01 14:56:10", str_format_date)
  d = d.replace(tzinfo=from_zone)
  d = d.astimezone(to_zone)
  d = datetime.strftime(d,'%d/%m/%Y %H:%M:%S')
  return d

In [45]:
def get_tweets(api,username):
  tweets={}
  for tweet in tqdm_notebook(tweepy.Cursor(api.user_timeline, screen_name = username).items()):
      tweets[tweet.id]= {
          'text': tweet.text,
          'created_at': convert_date(tweet.created_at),
          'author_name': tweet.user.name,
          'author_handler': str(tweet.user.screen_name),
          'user_description': tweet.user.description,
          'user_location': tweet.user.location,
          'user_followers_count': tweet.user.followers_count,
          'user_created_at': convert_date(tweet.user.created_at),
          'user_following_count': tweet.user.friends_count,
          'user_verified': tweet.user.verified,
          'lang': tweet.lang,
          'tweet_hashtags': tweet.entities['hashtags'],
          'mention_names': ["@" + d['screen_name'] for d in tweet.entities['user_mentions'] if 'screen_name' in d],
          'mention_ids': [d['id'] for d in tweet.entities['user_mentions'] if 'id' in d],
          'n_retweets': tweet.retweet_count,
          'n_favorites': tweet.favorite_count,
          'is_retweet': hasattr(tweet, 'retweeted_status')
      }
  df_tweets = pd.DataFrame.from_dict(tweets, orient='index')
  df_tweets['id'] = df_tweets.index
  return df_tweets
    #print(tweet.text)

#**Twitter**

Credenciales Consumo API

In [38]:
# API Keys para autenticacion
api_key = "UcZ9gR5yMV4MlKTFaSwhC1aNX" 
api_secret = "R5BjKcmHan6MGaX98UMhytQfLiZI7TLl9AATGtixJLwmAU2DFR"
access_token = "1452452402070794241-rKitK0leSslMs9XoXEDBTxOwplOGGU"
access_token_secret = "lBAxKokr4Hkg3DxvaUMTWkAqU0E9O5KfZCmJoPulnfSRn"

In [39]:
# Autorización para consumo API
auth = tweepy.OAuthHandler(api_key, api_secret) 
# Acceso al API
auth.set_access_token(access_token, access_token_secret) 
# Instancia para llamar a la API 
api = tweepy.API(auth, wait_on_rate_limit=True) 

In [43]:
#Obtener Tweets de cuenta
df_tweets=get_tweets(api,"juanmontan0")

0it [00:00, ?it/s]

In [44]:
df_tweets.head()

,text,created_at,author_name,author_handler,user_description,user_location,user_followers_count,user_created_at,user_following_count,user_verified,lang,tweet_hashtags,mention_names,mention_ids,n_retweets,n_favorites,is_retweet,id
1488526595614773253,Esto es una prueba para la tesis de grado,01/02/2022 09:56:10,juanmontan0,juanmontan0,,,0,01/02/2022 09:56:10,8,False,es,[],[],[],0,0,False,1488526595614773253
1488522887137046543,prueba 3,01/02/2022 09:56:10,juanmontan0,juanmontan0,,,0,01/02/2022 09:56:10,8,False,es,[],[],[],0,0,False,1488522887137046543
1488522868623355912,prueba 2,01/02/2022 09:56:10,juanmontan0,juanmontan0,,,0,01/02/2022 09:56:10,8,False,es,[],[],[],0,0,False,1488522868623355912
1488521198170513414,prueba,01/02/2022 09:56:10,juanmontan0,juanmontan0,,,0,01/02/2022 09:56:10,8,False,es,[],[],[],0,0,False,1488521198170513414


#**Cargar Corpus**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Corpus_Noticias_Falsas_Delimitado.csv to Corpus_Noticias_Falsas_Delimitado.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['Corpus_Noticias_Falsas_Delimitado.csv']))

In [ ]:
print(df.TEXT.iloc[0])

El presidente Iván Duque condenó fuertemente el atentado terrorista que se presentó en la noche de este viernes, 7 de enero, contra miembros del Escuadrón Móvil Antidisturbios (Esmad) en la ciudad de Cali.


In [ ]:
df=df.drop(['Nombre','Unnamed: 8','Unnamed: 9'], axis=1)

#**Preprocesamiento**

* Convertir Mayusculas a minusculas
* Eliminar signos de puntuación

In [ ]:
df['headline_Clean']=df['headline'].str.lower().str.translate(str.maketrans('','',string.punctuation)).str.replace("\r", " ")
df['TEXT_Clean']=df['TEXT'].str.lower().str.translate(str.maketrans('','',string.punctuation)).str.replace("\r", " ")

In [ ]:
print(df.TEXT_Clean.iloc[0])

el presidente iván duque condenó fuertemente el atentado terrorista que se presentó en la noche de este viernes 7 de enero contra miembros del escuadrón móvil antidisturbios esmad en la ciudad de cali


In [ ]:
df.head()

,ID,CATEGORIA,TOPIC,SOURCE,headline,TEXT,LINK,headline_Clean,TEXT_Clean
0,1,Noticia Real,Politico,El tiempo,Duque dice que Eln se adjudicó atentado y busc...,El presidente Iván Duque condenó fuertemente e...,https://www.eltiempo.com/politica/gobierno/eln...,duque dice que eln se adjudicó atentado y busc...,el presidente iván duque condenó fuertemente e...
1,2,Noticia Real,Politico,El espectador,"Gustavo Petro se reunirá con Pedro Sánchez, pr...","Gustavo Petro se reunirá con Pedro Sánchez, pr...",https://www.elespectador.com/politica/?utm_sou...,gustavo petro se reunirá con pedro sánchez pre...,gustavo petro se reunirá con pedro sánchez pre...
2,3,Noticia Real,Politico,El tiempo,Marta Lucía Ramírez se queja por servicio en e...,Marta Lucía Ramírez se queja por servicio en e...,https://www.eltiempo.com/politica/gobierno/mar...,marta lucía ramírez se queja por servicio en e...,marta lucía ramírez se queja por servicio en e...
3,4,Noticia Real,Politico,El espectador,"Elecciones de 2022 requieren mayor cuidado, po...","Elecciones de 2022 requieren mayor cuidado, po...",https://www.elespectador.com/politica/eleccion...,elecciones de 2022 requieren mayor cuidado por...,elecciones de 2022 requieren mayor cuidado por...
4,5,Noticia Real,Politico,El Nuevo Siglo,Procuraduría citó a mandatarios por fiestas en...,ANTE LA expansión de la variante ómicron en el...,https://www.elnuevosiglo.com.co/articulos/01-0...,procuraduría citó a mandatarios por fiestas en...,ante la expansión de la variante ómicron en el...
...,...,...,...,...,...,...,...,...,...
492,496,Noticia Real,Politico,El colombiano,Destruyen valla del Partido Comunes en Puerto ...,Los pasacalles de la campaña de Comunes en Pue...,https://www.elcolombiano.com/colombia/politica...,destruyen valla del partido comunes en puerto ...,los pasacalles de la campaña de comunes en pue...
493,497,Noticia Real,Politico,El colombiano,"“Cuando la corrupción llega, hay que buscar re...",En medio de uno de los momentos más difíciles ...,https://www.elcolombiano.com/colombia/politica...,“cuando la corrupción llega hay que buscar rev...,en medio de uno de los momentos más difíciles ...
494,498,Noticia Real,Politico,El colombiano,Este sábado empieza la inscripción de candidat...,Entre el sábado 29 de enero y el viernes 11 de...,https://www.elcolombiano.com/colombia/politica...,este sábado empieza la inscripción de candidat...,entre el sábado 29 de enero y el viernes 11 de...
495,499,Noticia Real,Politico,El colombiano,Corte “no vio a profundidad lo que se ha logra...,El presidente Iván Duque respondió a la decisi...,https://www.elcolombiano.com/colombia/respuest...,corte “no vio a profundidad lo que se ha logra...,el presidente iván duque respondió a la decisi...


* Remover stop words

In [ ]:
nltk.download('stopwords')
stop_words_esp = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
